In [9]:
import os
import json

recipes = {}
tags = {}


def add_connection(parent, child):
    if recipes.get(parent) == None:
        recipes[parent] = []

    if recipes.get(child) == None:
        recipes[child] = []

    recipes[parent].append(child)

def fix_name(name):
    return name[10:]

def load_tags(filename, tag_info):
    filename = filename.split(".json")[0]
    tags[filename] = [fix_name(name) for name in tag_info["values"]]

def parse_recipe(recipe):
    if recipe.get("result") == None: return

    result = recipe["result"]
    if not isinstance(result, str): result = result["item"]
    result = fix_name(result)

    if recipe.get("key"):
        for _, item in recipe["key"].items():
            
            if isinstance(item, list):
                for i in item:
                    i = fix_name(i["item"])
                    add_connection(result, i)
            elif item.get("tag"):
                tag = fix_name(item["tag"])
                for item in tags[tag]:
                    add_connection(result, item)
            else:
                item = fix_name(item["item"])
                add_connection(result, item)
    elif recipe.get("ingredient"):
        ingredients = recipe["ingredient"]

        if isinstance(ingredients, list):
            for item in recipe["ingredient"]:
                item = fix_name(item["item"])
                add_connection(result, item)
        else:
            for _, item in recipe["ingredient"].items():
                item = fix_name(item)
                add_connection(result, item)


# load tags first
tag_dir = "./tags"
for filename in os.listdir(tag_dir):
    file = os.path.join(tag_dir, filename)
    with open(file, "r") as recipe:
        load_tags(filename, json.load(recipe))

# load recipes
rec_dir = "./recipes"
for filename in os.listdir(rec_dir):
    file = os.path.join(rec_dir, filename)
    with open(file, "r") as recipe:
        parse_recipe(json.load(recipe))


In [10]:
import networkx as nx

G = nx.Graph()
G.add_nodes_from(recipes.keys())

for node, neighbors in recipes.items():
    for neighbor in neighbors:
        G.add_edge(node, neighbor)

nx.write_gexf(G, "mygraph.gexf")

In [13]:
def get_num_connections(target):
    connections = 0
    for _, neighbors in recipes.items():
        if target in neighbors:
            connections += 1

    return connections + len(recipes.get(target, []))

wood_types = [
    "acacia",
    "birch",
    "crimson",
    "dark_oak",
    "jungle",
    "oak",
    "spruce",
    "warped"
]


for wt in wood_types:
    wt = f"{wt}_planks"
    print(wt, get_num_connections(wt))



acacia_planks 47
birch_planks 47
crimson_planks 46
dark_oak_planks 47
jungle_planks 47
oak_planks 47
spruce_planks 47
warped_planks 46
